In [ ]:
import tensorflow as tf
from tensorflow import keras

import externalTensor as exT
import numpy as np
from sklearn.model_selection import KFold

import pandas as pd
from boxScore import boxScore
import winsound
import wandb
from wandb.keras import WandbCallback
# wandb.login()

: 

# Preparation of Data

In [2]:
years="2018-19"
stats="traditional"
# stats='advance'
box_score=boxScore(years,stats)

x_train, x_test, y_train, y_test=box_score.separation()
y_train=np.array(y_train)
y_test=np.array(y_test)
input_dimension=len(x_train.columns)

# Possible Hyperparameters

In [3]:
    
activation=['relu','sigmoid']
# number_neurons=[10,30,50,100,150,200]
number_neurons=[10,30]
# possible_learning_rate=[0.0001,0.001,0.01]
possible_learning_rate=[0.0001]
epochs=500
batch_size=150

# First 

In [6]:

kf = KFold(n_splits=10, random_state=1, shuffle=True)
best_model_array=[]
best_config_array=[]
for nN in number_neurons:
    best_config_tmp={'val_acc':0}
    for el in possible_learning_rate:
        for act in activation:
                # Initialize model
                model = exT.makeModelSimple(nN,act,input_dimension)
                
                # Instantiate an optimizer to train the model.
                optimizer = keras.optimizers.SGD(learning_rate=el)
                # Instantiate  loss function.
                loss_fn = tf.keras.losses.BinaryCrossentropy()
                config={
                    "learning_rate":el,
                    'num_neurons':nN,
                    'acti_fun':act,
                    "batch_size": 200,
                    "epochs": epochs,
                    "architecture": "OneHiddenLayer",
                    "dataset":years+"_"+stats
                }
               
                model.compile(optimizer=optimizer,loss=loss_fn,metrics=['accuracy'])

                config=exT.trainModelCrossValidation(config,x_train,y_train,
                    x_test, y_test, 
                    model,
                    kf,
                    epochs,
                    batch_size
                   
                )
                
                #Saved the best model based on the accuracy
                if(config['val_acc']>best_config_tmp['val_acc']):
                    bestModel=model
                    best_config_tmp=config
                        
                    
                
    print("Best model",best_config_tmp)
    best_model_array.append(bestModel)
    best_config_array.append(best_config_tmp)
    # f = open("saved_model_"+stats+"/summary.txt", "a")
    # f.write(tmpName+"\n")
    # f.close()



winsound.Beep(440,2500)

Best model {'learning_rate': 0.0001, 'num_neurons': 10, 'acti_fun': 'relu', 'batch_size': 200, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2018-19_traditional', 'loss': 0.4199, 'acc': 0.8219, 'val_loss': 0.6901, 'val_acc': 0.6}
Best model {'learning_rate': 0.0001, 'num_neurons': 30, 'acti_fun': 'relu', 'batch_size': 200, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2018-19_traditional', 'loss': 0.312, 'acc': 0.8815, 'val_loss': 0.7171, 'val_acc': 0.58}


In [ ]:
# Best model {'learning_rate': 0.0001, 'num_neurons': 10, 'acti_fun': 'sigmoid', 'batch_size': 200, 'epochs': 500, 'architecture': 'OneHiddenLayer', 
# 'dataset': '2018-19_traditional', 'loss': 0.2644, 'acc': 0.9037, 'val_loss': 0.9654, 'val_acc': 0.56}
# Best model {'learning_rate': 0.0001, 'num_neurons': 30, 'acti_fun': 'sigmoid', 'batch_size': 200, 'epochs': 500, 'architecture': 'OneHiddenLayer', '
# dataset': '2018-19_traditional', 'loss': 0.216, 'acc': 0.9199, 'val_loss': 1.0047, 'val_acc': 0.56}

In [13]:
count=0
for model,config in zip(best_model_array,best_config_array):
    run = wandb.init(project='Integration', config=config)
    run.name="Qua"+str(count)
    config = wandb.config
    
    history=model.fit(x_train,y_train,batch_size=batch_size,verbose=False,validation_data=(x_test, y_test),epochs=epochs)
    for epoch in range (epochs):

        wandb.log({'epochs': epoch,
            'loss': history.history['loss'][epoch],
            'acc': float(history.history['accuracy'][epoch]), 
            'val_loss': history.history['val_loss'][epoch],
            'val_acc':float(history.history['val_accuracy'][epoch])})
    count+=1
    run.finish()

acc,▃▁▁▁▅▃▆▅▆▆▆▆▆▆▆▆▅▅▆▆▅▅▅▅▅█▅▆▆▅▆▆▆▆█▆▆▅▆█
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,████▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▅▅▆▆▆▇▇▇▇▇▇██
acc,0.86833
epochs,499
loss,0.36301
val_acc,0.56
val_loss,0.70435


acc,▁▁▁▁▁▁▃▁▁▁▆▃▃▃▃█▆█▆████▆████████████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
acc,0.87333
epochs,499
loss,0.32421
val_acc,0.56
val_loss,0.70466


<!-- Best model {'learning_rate': 0.001, 'num_neurons': 10, 'acti_fun': 'relu', 'loss': 0.0193, 'acc': 0.9987, 'val_loss': 1.4627, 'val_acc': 0.6}
Best model {'learning_rate': 0.0001, 'num_neurons': 30, 'acti_fun': 'sigmoid', 'loss': 0.4884, 'acc': 0.9517, 'val_loss': 0.6466, 'val_acc': 0.64}
Best model {'learning_rate': 0.0001, 'num_neurons': 50, 'acti_fun': 'sigmoid', 'loss': 0.5163, 'acc': 0.9437, 'val_loss': 0.656, 'val_acc': 0.66}
Best model {'learning_rate': 0.0001, 'num_neurons': 100, 'acti_fun': 'relu', 'loss': 0.0628, 'acc': 0.9951, 'val_loss': 0.775, 'val_acc': 0.64}
Best model {'learning_rate': 0.001, 'num_neurons': 150, 'acti_fun': 'relu', 'loss': 0.0135, 'acc': 0.9998, 'val_loss': 1.4478, 'val_acc': 0.64}
Best model {'learning_rate': 0.0001, 'num_neurons': 200, 'acti_fun': 'relu', 'loss': 0.0487, 'acc': 1.0, 'val_loss': 0.8261, 'val_acc': 0.64} -->

# L2 Regolarization


In [ ]:

kf = KFold(n_splits=10, random_state=1, shuffle=True)
best_model_array=[]
possible_pL2=[0.0001,0.001,0.01]
for nN in number_neurons:
    best_config_tmp={'val_acc':0}
    for el in possible_learning_rate:
        for act in activation:
            for pl2 in possible_pL2:
                # Initialize model
                model = exT.makeModelL2(nN,act,input_dimension,pl2)
                
                # Instantiate an optimizer to train the model.
                optimizer = keras.optimizers.SGD(learning_rate=el)
                # Instantiate  loss function.
                loss_fn = tf.keras.losses.BinaryCrossentropy()
                config={
                    "learning_rate":el,
                    'num_neurons':nN,
                    'acti_fun':act,
                    'pL2':pl2
                }
                
                model.compile(optimizer=optimizer,loss=loss_fn,metrics=['accuracy'])

                config=exT.trainModelCrossValidation(config,x_train,y_train,
                    x_test, y_test, 
                    model,
                    kf,
                    500
                   
                )
                
                #Saved the best model based on the accuracy
                if(config['val_acc']>best_config_tmp['val_acc']):
                    bestModel=model
                    best_config_tmp=config
                        
                    
                
    print("Best model",best_config_tmp)
    best_model_array.append(bestModel)
    # f = open("saved_model_"+stats+"/summary.txt", "a")
    # tmpName=str(bestRes['num_neurons'])+"_"+str(bestRes['acti_fun'])+"_"+str(bestRes['learning_rate'])+"_LOSS_"+str(bestRes['loss'])+"_ACC_"+str(bestRes['acc'])+"_LOSSVAL_"+str(bestRes['val_loss'])+"_ACCVAL_"+str(bestRes['val_acc'])  
    # f.write(tmpName+"\n")
    # f.close()


# Dropout

In [ ]:

kf = KFold(n_splits=10, random_state=1, shuffle=True)
best_model_array=[]
possible_p=[0.5,0.6,65]
for nN in number_neurons:
    best_config_tmp={'val_acc':0}
    for el in possible_learning_rate:
        for act in activation:
            for p in possible_p:
                # Initialize model
                model = exT.makeModelDropout(nN,act,input_dimension,p)
                
                # Instantiate an optimizer to train the model.
                optimizer = keras.optimizers.SGD(learning_rate=el)
                # Instantiate loss function.
                loss_fn = tf.keras.losses.BinaryCrossentropy()
                config={
                    "learning_rate":el,
                    'num_neurons':nN,
                    'acti_fun':act,
                    'p-dropout':p
                }
                
                model.compile(optimizer=optimizer,loss=loss_fn,metrics=['accuracy'])

                config=exT.trainModelCrossValidation(config,x_train,y_train,
                    x_test, y_test, 
                    model,
                    kf,
                    500
                   
                )
                
                #Saved the best model based on the accuracy
                if(config['val_acc']>best_config_tmp['val_acc']):
                    bestModel=model
                    best_config_tmp=config
                        
                    
                
    print("Best model",best_config_tmp)
    best_model_array.append(bestModel)
    # f = open("saved_model_"+stats+"/summary.txt", "a")
    # tmpName=str(bestRes['num_neurons'])+"_"+str(bestRes['acti_fun'])+"_"+str(bestRes['learning_rate'])+"_LOSS_"+str(bestRes['loss'])+"_ACC_"+str(bestRes['acc'])+"_LOSSVAL_"+str(bestRes['val_loss'])+"_ACCVAL_"+str(bestRes['val_acc'])  
    # f.write(tmpName+"\n")
    # f.close()


# AUTOENCODERS ONE LEVEL

In [4]:


combination_of_neurons=[[20,10],[15,5]]
for comb in combination_of_neurons:
    best_config_tmp={'val_acc':0}
    for el in possible_learning_rate:
    
        autoencoder,encoder,decoder = exT.autoencoderOneLevel(input_dimension,comb[0],comb[1])
        loss_fn = tf.keras.losses.BinaryCrossentropy()               
        autoencoder.compile(optimizer="adam",loss=loss_fn,metrics=['accuracy'])
        config={
                "learning_rate":el,
                'num_neurons':comb,
                'acti_fun':"relu"
                
            }
        tik=autoencoder.fit(x_train,x_train,batch_size=200,verbose=False,epochs=50,validation_data=(x_test,x_test))
        config['val_acc']=tik.history['val_accuracy'][-1]
        if(config['val_acc']>best_config_tmp['val_acc']):
                    bestModel=autoencoder
                    best_config_tmp=config
                 
    print("Best model",best_config_tmp)
    # bestModelArray.append(bestModel)

Best model {'learning_rate': 0.01, 'num_neurons': [20, 10], 'acti_fun': 'relu', 'val_acc': 0.5}
Best model {'learning_rate': 0.001, 'num_neurons': [15, 5], 'acti_fun': 'relu', 'val_acc': 0.5}


In [ ]:
model = tf.keras.models.load_model('saved_model_'+stats+'/bestModel')

optimizer = keras.optimizers.SGD(learning_rate=el)
                # Instantiate  loss function.
loss_fn = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer,loss=loss_fn,metrics=['accuracy'])

# AUTOENCODERS SECOND LEVEL

In [7]:
combination_of_neurons=[[20,15,5],[15,10,5]]
for comb in combination_of_neurons:
    best_config_tmp={'val_acc':0}
    for el in possible_learning_rate:
    
        autoencoder,encoder,decoder = exT.autoencoderTwoLevels(input_dimension,comb[0],comb[1],comb[2])
        loss_fn = tf.keras.losses.BinaryCrossentropy()               
        autoencoder.compile(optimizer="adam",loss=loss_fn,metrics=['accuracy'])
        config={
                "learning_rate":el,
                'num_neurons':comb,
                'acti_fun':"relu"
                
            }
        tik=autoencoder.fit(x_train,x_train,batch_size=200,verbose=False,epochs=50,validation_data=(x_test,x_test))
        config['val_acc']=tik.history['val_accuracy'][-1]
        if(config['val_acc']>best_config_tmp['val_acc']):
                    bestModel=autoencoder
                    best_config_tmp=config
                 
    print("Best model",best_config_tmp)

Best model {'learning_rate': 0.001, 'num_neurons': [20, 15, 5], 'acti_fun': 'relu', 'val_acc': 0.5}
Best model {'learning_rate': 0.0001, 'num_neurons': [15, 10, 5], 'acti_fun': 'relu', 'val_acc': 0.5}


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
best_acc=-1
for el in range(2,10):
    clf=RandomForestClassifier(max_depth=el, random_state=42)
    clf.fit(x_train,y_train)
    config=clf.predict(x_test)
    count=0
    for x,y in zip(config,y_test):
        if(x[0]==y[0]):
            count+=1
    acc=count/len(x_test)
    if acc> best_acc:
        best_acc=acc
print(best_acc,el)

In [ ]:
para_y_train=[]
for el1 in y_train:
    tmp=1
    if(el1[0]==1):
        tmp=0
    para_y_train.append(tmp)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
best_acc=-1
for el in range(2,10):
    clf=RandomForestClassifier(n_estimators=100,max_depth=el, random_state=42)
    clf.fit(x_train,para_y_train)
    config=clf.predict(x_test)
    count=0
    for x,y in zip(config,y_test):
        if(x==y[1]):
            count+=1
    acc=count/len(x_test)
    if acc> best_acc:
        best_acc=acc
print(best_acc,el)

# ADA

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

best_acc=-1
clf = AdaBoostClassifier(n_estimators=100, random_state=42)
clf.fit(x_train,para_y_train)
config=clf.predict(x_test)
count=0
for x,y in zip(config,y_test):
    if(x==y[1]):
        count+=1
acc=count/len(x_test)
if acc> best_acc:
    best_acc=acc
print(best_acc)